<a href="https://colab.research.google.com/github/aswinaus/RAG_Multimodal/blob/main/RAG_Multi_Modal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unstructured --quiet

In [ ]:
!pip install Chroma --quiet

In [ ]:
!pip install langchain_chroma --quiet

In [ ]:
!pip install pdfminer.six --quiet

In [ ]:
!pip install "unstructured[all-docs]" --quiet

In [ ]:
!pip install langchain_community --quiet

In [ ]:
import os
import io
import re
import uuid
import base64
import shutil
import requests
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import HTML, display
from unstructured.partition.pdf import partition_pdf
from langchain_core.documents import Document
from langchain_text_splitters import CharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain.chains.llm import LLMChain, PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.retrievers.multi_vector import MultiVectorRetriever
from openai import OpenAI as OpenAI_vLLM
from langchain_community.llms.vllm import VLLMOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-large-en')

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Download Data
data_dir = '/content/drive/MyDrive' # Input a data dir path from your mounted Google Drive

In [ ]:
!pip install poppler-utils --quiet
!apt-get install poppler-utils


In [ ]:
# Extract elements from PDF
def extract_pdf_elements(path, fname):
    """
    Extract images, tables, and chunk text from a PDF file.
    path: File path, which is used to dump images (.jpg)
    fname: File name
    """
    return partition_pdf(
        filename=path + fname,
        #strategy='hi_res',
        extract_text_and_images=False,
        #extract_images_in_pdf=False,
        extract_tables=False,
        infer_table_structure=False,
        chunking_strategy="by_title",
        max_characters=4000,
        new_after_n_chars=3800,
        combine_text_under_n_chars=2000,
        image_output_dir_path=path,
    )

# Categorize elements by type
def categorize_elements(raw_pdf_elements):
    """
    Categorize extracted elements from a PDF into tables and texts.
    raw_pdf_elements: List of unstructured.documents.elements
    """
    tables = []
    texts = []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            texts.append(str(element))
    return texts, tables

In [ ]:
!pip install pymupdf --quiet
import pymupdf

In [ ]:
folder_path = f"{data_dir}/RAG/data/Form_Data/"
file_name = "12559_Aswin_Bhaskaran.pdf"
doc=pymupdf.open(folder_path+file_name)
for page in doc:
    text=page.get_text()
    print(text)

In [ ]:
!pip install unstructured_pytesseract --quiet
!pip install pytesseract --quiet
!pip install pdf2image --quiet
!pip install tiktoken
import pytesseract
pytesseract.pytesseract.tesseract_cmd = "/usr/local/lib/python3.10/dist-packages/tesseract"

In [ ]:
# File path
folder_path = f"{data_dir}/RAG/data/Form990/"
file_name = "Form990_Kingsford_High_School.pdf"
print(folder_path+file_name)

# Get elements
raw_pdf_elements = extract_pdf_elements(folder_path, file_name)

# Get text, tables
texts, tables = categorize_elements(raw_pdf_elements)

# Enforce a specific token size for texts
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 1000, chunk_overlap = 0
)
joined_texts = " ".join(texts)
texts_token = text_splitter.split_text(joined_texts)
print(texts)
print("No of Textual Chunks:", len(texts))
print("No of Table Elements:", len(tables))
print("No of Text Chunks after Tokenization:", len(texts_token))
